In [1]:
import tweepy
import bs4
import requests

import pyglobalgoals.pyglobalgoals as gg

In [2]:
url = "http://www.globalgoals.org/"
req = requests.get(url)
print(req)
print(sorted(dir(req)))

<Response [200]>
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__iter__', '__module__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [3]:
if not req.ok:
    raise Exception(req)
content = req.content
print(content[:20])

<!DOCTYPE html>
<htm


In [4]:
bs = bs4.BeautifulSoup(req.content)
print(bs.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Article" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   The Global Goals
  </title>
  <meta content="In September 2015, the United Nations are launching global goals, a series of ambitious targets to end extreme poverty and tackle climate change for everyone by 2030." name="description"/>
  <link href="http://m.globalgoals.org" media="handheld" rel="alternate"/>
  <link href="http://www.globalgoals.org/wp-content/themes/global-goals-theme/style.f3c2651da2df5227ec67ef701bac2385.css" rel="stylesheet"/>
  <link href="http://www.globalgoals.org/wp-content/themes/global-goals-theme/global-goals-favicon.ico" rel="shortcut icon"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!--[if lt IE 9]>
	    <script src="http://www.globalgoals.org/wp-content/themes/global-goals-theme/js/vendor/html5shiv.min.js"></script>
	    <link rel="stylesheet" type="text/css" href="http://www.globalgoals.org/wp-co

/home/wturner/-wrk/-ce27/pyglobalgoals/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
tiles = bs.find_all(class_='goal-tile-wrapper')
tiles

[<div class="goal-tile-wrapper">\n<a class="background-color--poverty goal-tile" href="http://www.globalgoals.org/global-goals/no-poverty/">\n<div class="goal-tile-content">\n<img alt="No Poverty Icon" class="goal--frontpage-icon" src="http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/no-poverty1.png"/>\n<div class="overlay">\n<h4 class="inverted">Read More</h4>\n</div>\n</div>\n</a>\n</div>,
 <div class="goal-tile-wrapper">\n<a class="background-color--hunger goal-tile" href="http://www.globalgoals.org/global-goals/no-hunger/">\n<div class="goal-tile-content">\n<img alt="Zero Hunger Icon" class="goal--frontpage-icon" src="http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/gg-2nohunger-english.png"/>\n<div class="overlay">\n<h4 class="inverted">Read More</h4>\n</div>\n</div>\n</a>\n</div>,
 <div class="goal-tile-wrapper">\n<a class="background-color--health goal-tile" href="http://www.globalgoals.org/global-go

In [6]:
tile = tiles[0]
tile

<div class="goal-tile-wrapper">\n<a class="background-color--poverty goal-tile" href="http://www.globalgoals.org/global-goals/no-poverty/">\n<div class="goal-tile-content">\n<img alt="No Poverty Icon" class="goal--frontpage-icon" src="http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/no-poverty1.png"/>\n<div class="overlay">\n<h4 class="inverted">Read More</h4>\n</div>\n</div>\n</a>\n</div>

In [7]:
link = tile.findNext('a')
img = link.findNext('img')
img_title = img['alt'][:-5]
img_src = img['src']
link_href = link['href']
{'name': img_title, 'img_src': img_src, 'href': link_href}

{'href': 'http://www.globalgoals.org/global-goals/no-poverty/',
 'img_src': 'http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/no-poverty1.png',
 'name': 'No Poverty'}

In [8]:
import collections
def get_data_from_goal_tile_wrapper_div(node, n=None):
    link = node.findNext('a')
    img = link.findNext('img')
    img_title = img['alt'][:-5]
    img_src = img['src']
    link_href = link['href']
    output = collections.OrderedDict({'@type': 'GlobalGoal'})
    if n:
        output['n'] = n
    output['name'] = img_title
    output['image'] = img_src
    output['url'] = link_href
    return output

def get_goal_tile_data(bs):
    for i, tile in enumerate(bs.find_all(class_='goal-tile-wrapper'), 1):
        yield get_data_from_goal_tile_wrapper_div(tile, n=i) 
        
tiles = list(get_goal_tile_data(bs))
import json
print(json.dumps(tiles, indent=2))

[
  {
    "@type": "GlobalGoal", 
    "n": 1, 
    "name": "No Poverty", 
    "image": "http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/no-poverty1.png", 
    "url": "http://www.globalgoals.org/global-goals/no-poverty/"
  }, 
  {
    "@type": "GlobalGoal", 
    "n": 2, 
    "name": "Zero Hunger", 
    "image": "http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/gg-2nohunger-english.png", 
    "url": "http://www.globalgoals.org/global-goals/no-hunger/"
  }, 
  {
    "@type": "GlobalGoal", 
    "n": 3, 
    "name": "Good Health & Well-Being", 
    "image": "http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/gg-3goodhealth-english.png", 
    "url": "http://www.globalgoals.org/global-goals/good-health/"
  }, 
  {
    "@type": "GlobalGoal", 
    "n": 4, 
    "name": "Quality Education", 
    "image": "http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcd

In [9]:
goal_tiles = tiles[:-1]
goal_tiles

[OrderedDict([('@type', 'GlobalGoal'), ('n', 1), ('name', 'No Poverty'), ('image', 'http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/no-poverty1.png'), ('url', 'http://www.globalgoals.org/global-goals/no-poverty/')]),
 OrderedDict([('@type', 'GlobalGoal'), ('n', 2), ('name', 'Zero Hunger'), ('image', 'http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/gg-2nohunger-english.png'), ('url', 'http://www.globalgoals.org/global-goals/no-hunger/')]),
 OrderedDict([('@type', 'GlobalGoal'), ('n', 3), ('name', 'Good Health & Well-Being'), ('image', 'http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/gg-3goodhealth-english.png'), ('url', 'http://www.globalgoals.org/global-goals/good-health/')]),
 OrderedDict([('@type', 'GlobalGoal'), ('n', 4), ('name', 'Quality Education'), ('image', 'http://012db3a8f7b65be46638-bcedd9d3bba1b1276c7bea0b3611599f.r67.cf2.rackcdn.com/2015/07/education.p